In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

In [2]:
format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

### Change kind en masse

In [3]:
kind = 'DOS'
div_year = 2021
div_quarter = 2
sell_year = 2021

### Dividends on specified year and quarter

In [4]:
sql = """
SELECT id, name, year, quarter, number, ppu, net AS div_amt, x_date, p_date
FROM dividends
WHERE year = %s AND quarter = %s
ORDER BY name, year DESC, quarter DESC
"""
sql = sql % (div_year, div_quarter)
dividends = pd.read_sql(sql, conpf)
#dividends.set_index(['name'],inplace=True)
dividends.style.format(format_dict)

,id,name,year,quarter,number,ppu,div_amt,x_date,p_date
0,251,ADVANC,2021,2,600,3.4500,"1,863.00",2021-08-16,2021-09-01
1,250,CPNCG,2021,2,"20,000",0.2450,"4,410.00",2021-08-10,2021-09-03
2,252,DIF,2021,2,"60,000",0.2610,"15,660.00",2021-08-10,2021-09-03
3,247,DTAC,2021,2,"6,000",1.0500,"5,670.00",2021-07-30,2021-08-16
4,266,GVREIT,2021,2,"10,000",0.1960,"1,764.00",2021-08-23,2021-09-08
5,256,JASIF,2021,2,"130,000",0.2400,"31,200.00",2021-08-20,2021-09-07
6,257,JMART,2021,2,"3,000",0.4000,"1,080.00",2021-08-23,2021-09-08
7,254,JMT,2021,2,"2,500",0.4000,900.00,2021-08-23,2021-09-06
8,267,KCE,2021,2,"3,600",0.6000,"2,080.80",2021-08-23,2021-09-08
9,262,LH,2021,2,"24,000",0.2500,"5,400.00",2021-08-24,2021-09-09


### Bind dividends to sales record process

In [5]:
sr = dividends["name"]
names = sr.values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ADVANC', 'CPNCG', 'DIF', 'DTAC', 'GVREIT', 'JASIF', 'JMART', 'JMT', 'KCE', 'LH', 'NOBLE', 'ORI', 'PTTEP', 'SAT', 'SINGER', 'STA', 'SUPEREIF', 'SYNEX', 'TQM', 'TU'"

In [6]:
sql = '''
SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE T.name IN (%s) 
AND S.dividend_id = 0
AND YEAR(s.date) = %s
ORDER BY T.name, S.date'''
sql = sql % (in_p, sell_year)
print(sql)


SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE T.name IN ('ADVANC', 'CPNCG', 'DIF', 'DTAC', 'GVREIT', 'JASIF', 'JMART', 'JMT', 'KCE', 'LH', 'NOBLE', 'ORI', 'PTTEP', 'SAT', 'SINGER', 'STA', 'SUPEREIF', 'SYNEX', 'TQM', 'TU') 
AND S.dividend_id = 0
AND YEAR(s.date) = 2021
ORDER BY T.name, S.date


In [7]:
sells = pd.read_sql(sql, conpf)
sells.sort_values(['name','sell_date'],ascending=[True,True]).style.format(format_dict)

,name,qty,sell_p,buy_p,diff,sell_amt,buy_amt,profit,sell_date,buy_date,days,pct,id,kind
0,DIF,"10,000",12.30,14.70,-2.40,"122,727.57","147,325.59","-24,598.02",2021-06-04,2019-05-15,751,-8.11,726,HD
1,DIF,"10,000",12.40,14.70,-2.30,"123,725.35","147,325.59","-23,600.24",2021-06-04,2019-05-15,751,-7.79,727,HD
2,DIF,"10,000",12.50,14.70,-2.20,"124,723.14","147,325.59","-22,602.45",2021-06-08,2019-05-15,755,-7.42,731,HD
3,DIF,"10,000",12.60,14.70,-2.10,"125,720.92","147,325.59","-21,604.67",2021-06-09,2019-05-15,756,-7.08,732,HD
4,DTAC,"2,000",36.50,31.50,5.00,"72,838.31","63,139.54","9,698.77",2021-07-30,2021-07-19,11,509.70,772,DTD
5,DTAC,"2,000",36.50,31.50,5.00,"72,838.31","63,139.54","9,698.77",2021-07-30,2021-07-19,11,509.70,773,DTD
6,DTAC,"2,000",37.00,31.50,5.50,"73,836.10","63,139.54","10,696.56",2021-08-02,2021-07-27,6,"1,030.59",775,DTD
7,JMART,"3,000",35.75,32.50,3.25,"107,012.45","97,715.96","9,296.49",2021-08-19,2021-08-02,17,204.27,798,DTD
8,JMART,"3,000",37.00,32.50,4.50,"110,754.15","97,715.96","13,038.19",2021-08-30,2021-08-11,19,256.33,802,DTD
9,LH,"6,000",8.05,10.45,-2.40,"48,193.02","62,838.88","-14,645.86",2021-07-02,2018-08-23,"1,044",-8.15,747,DIV


In [8]:
df_merge = pd.merge(sells, dividends, on='name', how='inner')
df_merge['days'] = df_merge.sell_date - df_merge.x_date
df_merge['days'] = df_merge['days']/np.timedelta64(1,'D')
df_merge['days'] = df_merge['days'].astype(int)
mask_days = df_merge.days >= 0
df_merge[mask_days]

,name,qty,sell_p,buy_p,diff,sell_amt,buy_amt,profit,sell_date,buy_date,...,id_x,kind,id_y,year,quarter,number,ppu,div_amt,x_date,p_date
4,DTAC,2000,36.5,31.5,5.0,72838.31,63139.54,9698.77,2021-07-30,2021-07-19,...,772,DTD,247,2021,2,6000,1.05,5670.0,2021-07-30,2021-08-16
5,DTAC,2000,36.5,31.5,5.0,72838.31,63139.54,9698.77,2021-07-30,2021-07-19,...,773,DTD,247,2021,2,6000,1.05,5670.0,2021-07-30,2021-08-16
6,DTAC,2000,37.0,31.5,5.5,73836.10,63139.54,10696.56,2021-08-02,2021-07-27,...,775,DTD,247,2021,2,6000,1.05,5670.0,2021-07-30,2021-08-16
8,JMART,3000,37.0,32.5,4.5,110754.15,97715.96,13038.19,2021-08-30,2021-08-11,...,802,DTD,257,2021,2,3000,0.40,1080.0,2021-08-23,2021-09-08
20,ORI,5000,9.6,9.4,0.2,47893.68,47104.10,789.58,2021-08-27,2018-11-13,...,801,DTD,263,2021,2,25000,0.12,2700.0,2021-08-24,2021-09-09


In [9]:
cols = 'name qty sell_p buy_p buy_date x_date sell_date days id_x kind id_y number ppu'.split()

In [10]:
df_dsp = df_merge[cols]
mask_name = df_dsp.name == 'WHART'
df_dsp[mask_name]

,name,qty,sell_p,buy_p,buy_date,x_date,sell_date,days,id_x,kind,id_y,number,ppu


### dividend_id = id_y, id = id_x 

In [11]:
sql = '''
UPDATE sells
SET dividend_id = 245
WHERE id = 720'''
rp = conpf.execute(sql)
rp.rowcount

1

### End of Bind sell to dividend record process

In [12]:
cols = 'name buy_date p_date qty buy_amt div_amt days pct'.split()
df_merge[cols].style.format(format_dict)

,name,buy_date,p_date,qty,buy_amt,div_amt,days,pct
0,DIF,2019-05-15,2021-06-04,"10,000","147,325.59","26,100.00",23,-8.11
1,DIF,2019-05-15,2021-06-04,"10,000","147,325.59","26,100.00",23,-7.79
2,DIF,2019-05-15,2021-06-04,"10,000","147,325.59","26,100.00",27,-7.42
3,DIF,2019-05-15,2021-06-04,"10,000","147,325.59","26,100.00",28,-7.08
4,WHART,2021-05-27,2021-06-14,"9,000","119,965.12","1,551.15",0,111.90
5,WHART,2021-06-01,2021-06-14,"10,000","130,287.94","1,551.15",5,677.25


### Percent calculation of sells after dividend payment date

In [13]:
sql = """
SELECT name, number, ppu, D.net AS div_amt, profit, profit + D.net AS ttl_amt, B.net AS buy_amt,  
p_date, S.date AS sell_date, B.date AS buy_date, DATEDIFF(IF(p_date > S.date, p_date, S.date),B.date) AS days,
(profit + D.net)/B.net*36500/DATEDIFF(IF(p_date > S.date, p_date, S.date),B.date) AS pct, year, quarter
FROM sells S 
JOIN buys B ON S.buy_id = B.id
JOIN dividends D
ON dividend_id = D.id
ORDER BY name
"""
sells_a_div = pd.read_sql(sql, conpf)
sells_a_div.sort_values(['p_date'],ascending=[False]).style.format(format_dict)

,name,number,ppu,div_amt,profit,ttl_amt,buy_amt,p_date,sell_date,buy_date,days,pct,year,quarter
67,WHART,"9,000",0.1915,"1,551.15",367.77,"1,918.92","119,965.12",2021-06-14,2021-05-28,2021-05-27,18,32.44,2021,1
50,SUPEREIF,"10,000",0.2670,"2,670.00",514.94,"3,184.94","109,241.42",2021-06-09,2021-06-08,2021-05-12,28,38.01,2021,1
38,POPF,"10,000",0.2470,"2,223.00",514.94,"2,737.94","109,241.42",2021-06-04,2021-06-07,2021-05-06,32,28.59,2021,1
65,UTP,"6,000",0.3900,"2,340.00",724.23,"3,064.23","107,036.56",2021-05-25,2021-03-22,2021-03-05,81,12.90,2020,4
44,ROJNA,"20,000",0.2000,"3,600.00","6,843.86","10,443.86","31,820.32",2021-05-24,2021-05-06,2019-09-19,613,19.54,2020,4
22,GUNKUL,"30,000",0.1820,"4,914.00","14,448.06","19,362.06","27,059.80",2021-05-24,2021-04-29,2020-06-15,343,76.14,2020,4
21,GUNKUL,"30,000",0.1820,"4,914.00","12,851.60","17,765.60","27,059.80",2021-05-24,2021-04-02,2020-06-15,343,69.86,2020,4
41,ROJNA,"20,000",0.2000,"3,600.00","1,356.03","4,956.03","31,820.32",2021-05-24,2021-04-26,2019-12-09,532,10.69,2020,4
43,ROJNA,"20,000",0.2000,"3,600.00","16,041.52","19,641.52","95,460.97",2021-05-24,2021-04-27,2019-11-29,542,13.86,2020,4
42,ROJNA,"20,000",0.2000,"3,600.00","6,095.51","9,695.51","31,820.32",2021-05-24,2021-05-10,2019-09-03,629,17.68,2020,4


In [14]:
total_profit = round(sells_a_div.ttl_amt.sum(),2)
buy_cost = round(sells_a_div.buy_amt.sum(),2)
avg_day = sells_a_div.days.mean()
sell_pct = round(total_profit/buy_cost*36500/avg_day,2)
total_profit,buy_cost,sell_pct

(318614.76, 4527996.84, 8.82)